In [1]:
import sys
#sys.path.insert(1, '/home/linus/projects/fdl/crisp')

In [2]:
import os
import pandas as pd
if os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [3]:
from models.SimpleNonLinearInvariantRiskMinimization import SimpleNonLinearInvariantRiskMinimization
from dataio.DataFrameDataset import *
from dataio.datasets import *

In [4]:
from synthetic.facebook_synthetic_data_generator import generator_example

dim_inv = 6
dim_spu = 60
n_exp_train = 1000 #int(2e2)
n_exp_test = 300
n_env = 5
save_dir = 'data/synthetic'

In [5]:
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": 1,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        #'dataset_fp': '../data/test_multiclass.pkl',
        'dataset_fp' : None,
        'output_data_regime' : ["real-valued", "binary", "binary", "real-valued", "multi-class"],
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
 #        'predictors': ['All'],
        'predictors': None,
        'environments': ['env_split'],
        'exclude': ['Subj_ID'],
        'synthetic_train_test_split' : True,
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": 0,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ERM", "RF", "ICP", "IRM", "DCF", "ITE", "LIRM", "NLICP"]
    },
    "use_cloud":False,
    "results_directory": "results/"
}
data_config = config['data_options']

In [11]:
LIRM_options = config.get("ensemble_options").get('LIRM', {})
LIRM_args = {
            "use_icp_initialization": False,
            "verbose": 0,
            "n_iterations": 100, # 1000
            "seed": 0,
            "lr": 0.005, # 0.001
            "cuda": False
        }
LIRM_args.update(LIRM_options)

In [12]:
def get_dataset_for_synthetic_experiment(n_example, dim_inv, dim_spu, n_exp_train, n_exp_test, n_env, save_dir):
    # training data
    generator_example(n_example, dim_inv, dim_spu, n_exp_train, n_env ,save_dir, False)
    outfile_train = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_train, n_env, False))

    # test data
    generator_example(n_example, dim_inv, dim_spu, n_exp_test, n_env ,save_dir, True)
    outfile_test = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_test, n_env, True))
    
    return outfile_train, outfile_test

In [13]:
df_train

,Causal_0,Causal_1,Causal_2,Causal_3,Causal_4,Causal_5,Non_causal_0,Non_causal_1,Non_causal_2,Non_causal_3,...,Non_causal_53,Non_causal_54,Non_causal_55,Non_causal_56,Non_causal_57,Non_causal_58,Non_causal_59,env_split,Subj_ID,Target
0,-0.000561,0.145849,0.225573,0.122880,-0.048546,0.045357,-0.970126,-1.064057,-0.757917,2.136059,...,0.253969,-1.254056,0.673715,1.387798,-0.409082,-0.392847,0.482796,0.0,0.0,0.342489
1,0.135144,0.043393,-0.055957,0.043434,0.027566,0.010969,-1.083471,-1.213974,0.670480,-4.095187,...,-1.430859,0.184115,0.624684,2.087685,0.766855,-0.830979,-0.343249,0.0,1.0,-0.369267
2,0.035942,-0.075374,0.022940,-0.025444,0.040704,0.125165,0.805213,0.394979,-0.951211,0.463409,...,-0.385191,-1.884077,0.282484,1.031757,1.301110,2.845189,-0.644065,0.0,2.0,-0.008399
3,-0.119911,-0.002569,0.180238,-0.105965,0.340283,-0.056867,-0.585122,1.133821,-0.501122,-0.611285,...,1.362050,0.977686,-1.760463,-0.527005,1.332132,-1.997353,-0.641273,0.0,3.0,-0.254743
4,0.074002,-0.071610,-0.020441,-0.031641,0.129374,0.134526,-1.256257,-0.377510,-2.115434,-0.075239,...,-0.440069,-1.864138,0.579406,-2.421886,0.800720,-0.032827,-0.270851,0.0,4.0,-0.485479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.077538,1.349056,-1.415919,1.840771,3.518825,2.799105,-0.931326,0.762403,0.083186,0.749913,...,-1.163051,-0.507593,-0.246181,1.543768,-0.775012,0.531035,-0.823280,4.0,995.0,-3.850351
996,-2.665901,0.748542,-3.082372,-1.095816,1.985437,4.636995,-2.057844,1.323252,0.044714,1.063649,...,-0.701056,0.116659,1.285114,1.082809,-0.428213,1.070121,0.296680,4.0,996.0,-2.234435
997,0.796232,3.511699,-0.188452,-1.421263,-1.700785,2.423992,-1.118468,0.531102,-1.395911,0.539168,...,-0.502925,0.072475,-1.197326,1.050143,-1.398511,-1.423200,-2.661162,4.0,997.0,10.327812
998,-2.382634,-0.371572,0.511012,1.624738,-1.620211,2.351675,1.155782,-1.299769,-0.530284,-0.725726,...,1.063071,0.998619,-0.164660,-1.457753,1.713706,0.165963,-0.516912,4.0,998.0,-4.578163


In [14]:
results = {}

for n_example in [1]:#,2,3,4,5]:
    output_data_regime = config["data_options"]["output_data_regime"][n_example-1]
    print("\n\nn_example:", n_example, "output_data_regime:", output_data_regime, "\n\n")
    
    outfile_train, outfile_test = get_dataset_for_synthetic_experiment(n_example, dim_inv, dim_spu, n_exp_train, n_exp_test, n_env, save_dir)
    df_train = pd.read_pickle(outfile_train)
    df_test = pd.read_pickle(outfile_test)

    config["data_options"]["dataset_fp_train"] = outfile_train
    config["data_options"]["dataset_fp_test"] = outfile_test
    config["data_options"]["predictors"] = list(df_train.columns[0:dim_inv+dim_spu])
    environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

    LIRM_args["target"] = config["data_options"]["targets"]
    LIRM_args["output_data_regime"] = output_data_regime
    LIRM_args["columns"] = test_dataset.predictor_columns
    LIRM_args['output_dim'] = 1

    lirm = SimpleNonLinearInvariantRiskMinimization(environment_datasets, val_dataset, test_dataset, LIRM_args)
    
    lirm_results_dict = lirm.results()
    results[n_example] = lirm_results_dict



n_example: 1 output_data_regime: real-valued 


Environments variables: {'E0': 0.1, 'E1': 1.5, 'E2': 2, 'E3': 0.30815539086172433, 'E4': 2.016811860476012}
Generated Synthetic Data according to the Facebook setup Example: 1
     df with  66  columns
Environments variables: {'E0': 0.1, 'E1': 1.5, 'E2': 2, 'E3': 0.30815539086172433, 'E4': 2.016811860476012}
Generated Synthetic Data according to the Facebook setup Example: 1
     df with  66  columns
Running a per sample experiment
Using synthetic dataset train/test split
Loaded  5  train environments
Env  0  has  200  samples
X shape  (200, 66)  y shape  (200, 1)
Env  1  has  200  samples
X shape  (200, 66)  y shape  (200, 1)
Env  2  has  200  samples
X shape  (200, 66)  y shape  (200, 1)
Env  3  has  200  samples
X shape  (200, 66)  y shape  (200, 1)
Env  4  has  200  samples
X shape  (200, 66)  y shape  (200, 1)
Loaded test set, X shape: (300, 66)  y shape:  (300, 1)
Start simple nonlinear-IRM training procedure
input_gradients: torc

In [11]:
output_data_regime

'real-valued'

In [9]:
print(results.get(n_example).keys())
"acc/r2:", results.get(n_example).get("test_acc")

dict_keys(['test_logits', 'test_acc', 'test_nll', 'test_probs', 'test_labels', 'feature_coeffients', 'to_bucket'])


('acc/r2:', -1915.978625041141)

In [10]:
result = results.get(n_example).get("to_bucket")

if output_data_regime == "multi-class":
    r = sorted(zip(result.get("features"), np.abs(result.get("coefficients")).sum(axis=1)), key=lambda x: x[1], reverse=True)
else:
    r = sorted(zip(result.get("features"), result.get("coefficients")), key=lambda x: abs(x[1]), reverse=True)
r

[('Non_causal_3', 0.020613588392734528),
 ('Non_causal_16', 0.020209843292832375),
 ('Non_causal_10', 0.02005128748714924),
 ('Non_causal_11', 0.01926610991358757),
 ('Non_causal_33', 0.01868005283176899),
 ('Non_causal_0', 0.018617410212755203),
 ('Causal_5', 0.018483629450201988),
 ('Non_causal_40', 0.01843417063355446),
 ('Non_causal_26', 0.018231097608804703),
 ('Non_causal_57', 0.018154120072722435),
 ('Non_causal_14', 0.017951887100934982),
 ('Non_causal_24', 0.0179157517850399),
 ('Non_causal_37', 0.017474452033638954),
 ('Non_causal_23', 0.01736176200211048),
 ('Non_causal_44', 0.01697571761906147),
 ('Non_causal_18', 0.016747891902923584),
 ('Non_causal_35', 0.016263213008642197),
 ('Non_causal_13', 0.01620437391102314),
 ('Non_causal_17', 0.016199272125959396),
 ('Non_causal_4', 0.016169942915439606),
 ('Non_causal_6', 0.01597842574119568),
 ('Non_causal_41', 0.015846094116568565),
 ('Non_causal_1', 0.015569218434393406),
 ('Non_causal_49', 0.015395441092550755),
 ('Non_causa